Придумайте эвристику для предсказания целевого класса. На данном этапе нельзя использовать ML, допускаются исключительно наивные методы.
Реализуйте один из методов векторизации текста и обучите одну из моделей классического ML для данной задачи. Можно использовать любые библиотеки.

### Предсказание целевого класса с использованием наивных методов

In [2]:
import pandas as pd
from collections import Counter
import re

In [3]:
# Загрузка данных
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [4]:
# Основные статистики
print(train_df.describe())

       Unnamed: 0                                               Text Sentiment
count       41158                                              41158     41155
unique      41158                                              41158         5
top             0  @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...  Positive
freq            1                                                  1     11422


In [5]:
# Распределение целевого класса
print(train_df['Sentiment'].value_counts())

Sentiment
Positive              11422
Negative               9917
Neutral                7711
Extremely Positive     6624
Extremely Negative     5481
Name: count, dtype: int64


Далее проведем предобработку текста, в которую будет входить:  В вашем текущем коде уже реализованы следующие шаги нормализации текста:
замена `NaN` на пустые строки; преобразование всех значений в строки; преобразование текста в нижний регистр; замена табуляции на пробел; удаление знаков препинания, ссылок, и цифр; удаление двойных пробелов; частотный анализ слов и удаление самых частотных слов.

In [6]:

# заменяем NaN на пустые строки
train_df["Text"] = train_df["Text"].fillna("")

# преобразуем все значения в строки (на случай если там есть не строковые данные)
train_df["Text"] = train_df["Text"].astype(str)

# преобразование текста в нижний регистр
train_df["Text"] = train_df["Text"].str.lower()

# замена табуляции на пробел
train_df["Text"] = train_df["Text"].str.replace(r'\t', ' ', regex=True)



# удаляем знаки препинания, ссылки и цифры
# удаляем ссылки из текста
train_df["Text"] = train_df["Text"].str.replace(
    r'http\S+|www.\S+', '', regex=True)
train_df["Text"] = train_df["Text"].str.replace(
    r'[^\w\s]', '', regex=True)  # Удаляем знаки препинания
train_df["Text"] = train_df["Text"].str.replace(
    r'\d', '', regex=True)  # Удаляем цифры
# удаление двойных пробелов
train_df["Text"] = train_df["Text"].str.replace(r'\s+', ' ', regex=True)

# частотный анализ слов
words_counter = Counter(" ".join(train_df["Text"]).split()).most_common()

# выводим 10 самых часто встречающихся слов
print(words_counter[:10])

[('the', 44708), ('to', 38330), ('and', 23973), ('of', 21509), ('a', 19324), ('in', 19119), ('covid', 18529), ('coronavirus', 17968), ('for', 14035), ('is', 12241)]


In [7]:
# Предполагается, что train_df уже определен и содержит колонку "Text"

# Частотный анализ слов
#words_counter = Counter(" ".join(train_df["Text"]).split()).most_common(10)

# Получаем список 10 самых частотных слов
most_common_words = [word for word, count in words_counter[:10]]
print(most_common_words)
# Функция для удаления самых частотных слов из текста


def remove_most_common_words(text):
    return " ".join([word for word in text.split() if word not in most_common_words])


# Удаление 10 самых частотных слов из каждого текста
train_df["Text"] = train_df["Text"].apply(remove_most_common_words)

# Проверяем результат
print(train_df["Text"].head())

['the', 'to', 'and', 'of', 'a', 'in', 'covid', 'coronavirus', 'for', 'is']
0                         menyrbie phil_gahan chrisitv
1    advice talk your neighbours family exchange ph...
2    australia woolworths give elderly disabled ded...
3    my food stock not only one which empty please ...
4    me ready go at supermarket during outbreak not...
Name: Text, dtype: object


In [8]:
# Определение ключевых слов для каждого класса
keywords = {
    "Positive": [
        "safe", "support", "relief", "help", "hope", "recover",
        "good", "positive", "improvement", "secure", "trust",
        "benefit", "happy", "calm", "peace"
    ],
    "Negative": [
        "panic", "shortage", "danger", "risk", "threat", "fear",
        "problem", "negative", "loss", "worry", "unsecure",
        "collapse", "failure", "frustration", "stress", "harm",
        "crisis", "doubt", "uncertain"
    ],
    "Neutral": [
        "information", "update", "report", "notice", "announcement",
        "status", "fact", "data", "statement", "average", "neutral",
        "balance", "comment", "opinion", "unbiased", "objective"
    ],
    "Extremely Positive": [
        "amazing", "incredible", "fantastic", "wonderful", "perfect",
        "brilliant", "awesome", "outstanding", "excellent",
        "superb", "life-saving", "remarkable", "exceptional",
        "phenomenal", "extraordinary", "miracle", "best"
    ],
    "Extremely Negative": [
        "disaster", "horrible", "awful", "terrible", "catastrophe",
        "devastating", "ruin", "nightmare", "tragic", "unbearable",
        "horrific", "worst", "dreadful", "abysmal", "failure",
        "chaos", "collapse", "irreparable", "destruction"
    ]
}

In [9]:
# Функция для предсказания
def predict_sentiment(text):
    sentiment_scores = {key: 0 for key in keywords.keys()}
    for word in text.split():
        for sentiment, key_words in keywords.items():
            if word in key_words:
                sentiment_scores[sentiment] += 1
    return max(sentiment_scores, key=sentiment_scores.get)

In [10]:
# Пример использования функции
for text in train_df["Text"]:
    #text = "Your safety is our priority. Please stay calm and do not panic."
    print(predict_sentiment(text))

Positive
Positive
Positive
Positive
Negative
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Neutral
Positive
Positive
Positive
Positive
Positive
Negative
Positive
Negative
Positive
Positive
Negative
Positive
Negative
Positive
Neutral
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Negative
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Positive
Negative
Positive
Negative
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Negative
Negative
Negative
Positive
Positive
Negative
Positive
Positive
Negative
Neutral
Positive
Negative
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Negative
Positive
Negative
Positive
Negative
Positive
Neutral
Positive
Posit

In [11]:
# Список предсказанных меток
predictions = [predict_sentiment(text) for text in train_df["Text"]]

# Истинные метки
true_labels = train_df["Sentiment"].tolist()

# Сравнение предсказаний с истинными метками
correct_predictions = sum(pred == true for pred,
                          true in zip(predictions, true_labels))
total_predictions = len(true_labels)

# Вычисление точности
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.31


### Построение модели ML 

вариант 1

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

lemmatizer = WordNetLemmatizer()

# скачиваем ресурсы NLTK
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Anna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

В дополнение к предобработке текста, добавляем следующие шаги: удаление стоп-слов(не несущих значимой информации) и лемматизация (приведение слов к их базовой/основной форме).

In [45]:
# Определение функции предобработки текста
def preprocess_text(text):
    # Токенизация
    tokens = word_tokenize(text)
    # Удаление стоп-слов и лемматизация
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.isalpha(
    ) and token.lower() not in stopwords.words('english')]
    return ' '.join(tokens)

In [48]:

train_df["Text"] = train_df["Text"].apply(preprocess_text)

логистическая регрессия

In [49]:
# векторизация текста
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_df["Text"])

In [50]:
# разбиение данных на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(
    X, true_labels, test_size=0.2, random_state=42)

In [57]:
# обучение модели вариант 1 - лог регрессия
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

LogisticRegression(max_iter=500)

In [58]:
# предсказание на валидационной выборке
predictions_val = model.predict(X_val)

In [60]:
# 5. Вычисление метрики
accuracy_val = accuracy_score(y_val, predictions_val)
print(f"Validation Accuracy: {accuracy_val:.2f}")

Validation Accuracy: 0.56


LSTM

In [61]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D

In [73]:
# высказывания, эмоциональную окраску которых надо определить содержит колонка 'Text' и 'Sentiment'- метки классов
texts = train_df['Text'].values
labels = train_df['Sentiment'].values

# токенизация проводится по словам, токенайзер,будет учитывать только 5000 самых частых слов из текста

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# паддинг последовательностей(выравнивание)
max_sequence_length = max(len(x) for x in sequences)
data = pad_sequences(sequences, maxlen=max_sequence_length)

# Преобразование текстовых меток в числовые
label_encoder = LabelEncoder()
# Преобразуем текстовые метки в числовые
labels = label_encoder.fit_transform(labels)

# Преобразование меток в категориальный формат
labels = to_categorical(np.asarray(labels))

#разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42)

In [74]:
# Создание модели LSTM, настройка нейронной сети: размерность векторного пространства - 100; регуляризация через SpatialDropout(0,2);
# многоклассовая классификация через полносвязный слой с активацией softmax.

embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(labels.shape[1], activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 36, 100)           500000    
                                                                 
 spatial_dropout1d_1 (Spati  (None, 36, 100)           0         
 alDropout1D)                                                    
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense_1 (Dense)             (None, 6)                 606       
                                                                 
Total params: 581006 (2.22 MB)
Trainable params: 581006 (2.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [75]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучение модели
history = model.fit(X_train, y_train, batch_size=64, epochs=5, validation_data=(X_test, y_test), verbose=2)

Epoch 1/5
515/515 - 29s - loss: 1.1986 - accuracy: 0.4929 - val_loss: 0.8784 - val_accuracy: 0.6718 - 29s/epoch - 55ms/step
Epoch 2/5
515/515 - 25s - loss: 0.7715 - accuracy: 0.7185 - val_loss: 0.7905 - val_accuracy: 0.7202 - 25s/epoch - 48ms/step
Epoch 3/5
515/515 - 26s - loss: 0.6851 - accuracy: 0.7597 - val_loss: 0.7841 - val_accuracy: 0.7246 - 26s/epoch - 51ms/step
Epoch 4/5
515/515 - 26s - loss: 0.6372 - accuracy: 0.7790 - val_loss: 0.8130 - val_accuracy: 0.7159 - 26s/epoch - 51ms/step
Epoch 5/5
515/515 - 26s - loss: 0.5949 - accuracy: 0.7979 - val_loss: 0.8176 - val_accuracy: 0.7138 - 26s/epoch - 51ms/step
